<a href="https://colab.research.google.com/github/kregmi001/ML-and-Deep-Learning-for-Single-and-Muti-Label-Classification/blob/main/Bert_Encoding_for_Multi_Class_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import ast
from sklearn.model_selection import train_test_split
import tensorflow as tf


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
movies_v2 = pd.read_csv('/content/drive/My Drive/bertmodel/tmdb_5000_movies.csv')  
credits_v2 = pd.read_csv('/content/drive/My Drive/bertmodel/tmdb_5000_credits.csv')
movies_v2 = movies_v2.merge(credits_v2,on='title')


movies_clean = movies_v2[['title','keywords','cast','popularity','overview','genres']]
movies_clean.dropna(inplace=True)
movies_clean.head(5)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,title,keywords,cast,popularity,overview,genres
0,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...",150.437577,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."
1,Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...",139.082615,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""..."
2,Spectre,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...",107.376788,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."
3,The Dark Knight Rises,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...",112.312950,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam..."
4,John Carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...",43.926995,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."


In [4]:
def convert_1(text):
    arr = []
    for i in ast.literal_eval(text):
            arr.append(i['name'])
    return arr

movies_clean['genres'] = movies_v2['genres'].apply(convert_1)

movies_clean['genres'] = movies_clean['genres'].apply(lambda x: ", ".join(x))


movies_summary = movies_clean[['title','overview','genres']]
movies_summary = movies_summary.dropna(subset=['title','overview','genres'])

movies_summary.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,title,overview,genres
0,Avatar,"In the 22nd century, a paraplegic Marine is di...","Action, Adventure, Fantasy, Science Fiction"
1,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","Adventure, Fantasy, Action"
2,Spectre,A cryptic message from Bond’s past sends him o...,"Action, Adventure, Crime"
3,The Dark Knight Rises,Following the death of District Attorney Harve...,"Action, Crime, Drama, Thriller"
4,John Carter,"John Carter is a war-weary, former military ca...","Action, Adventure, Science Fiction"


In [5]:
import re
def clean_text(text):
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
    text = text.lower() 
    
    return text

movies_summary['clean_tag'] = movies_summary['overview'].apply(lambda x: clean_text(x))



movies_summary_clean_1 = movies_summary[['title','clean_tag','genres']]

movies_summary_clean_1.head(10)

,title,clean_tag,genres
0,Avatar,in the nd century a paraplegic marine is dispa...,"Action, Adventure, Fantasy, Science Fiction"
1,Pirates of the Caribbean: At World's End,captain barbossa long believed to be dead has ...,"Adventure, Fantasy, Action"
2,Spectre,a cryptic message from bond s past sends him o...,"Action, Adventure, Crime"
3,The Dark Knight Rises,following the death of district attorney harve...,"Action, Crime, Drama, Thriller"
4,John Carter,john carter is a war weary former military cap...,"Action, Adventure, Science Fiction"
5,Spider-Man 3,the seemingly invincible spider man goes up ag...,"Fantasy, Action, Adventure"
6,Tangled,when the kingdoms most wanted and most charmin...,"Animation, Family"
7,Avengers: Age of Ultron,when tony stark tries to jumpstart a dormant p...,"Action, Adventure, Science Fiction"
8,Harry Potter and the Half-Blood Prince,as harry begins his sixth year at hogwarts he ...,"Adventure, Fantasy, Family"
9,Batman v Superman: Dawn of Justice,fearing the actions of a god like super hero l...,"Action, Adventure, Fantasy"


In [6]:
def remove_spaces(x):
    x=x.split(",")
    nospace=[]
    for item in x:
        item=item.lstrip()
        nospace.append(item)
    return (",").join(nospace)

movies_summary_clean_1['genres']=movies_summary_clean_1['genres'].apply(remove_spaces)
movies_summary_clean_1['genres']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


0       Action,Adventure,Fantasy,Science Fiction
1                       Adventure,Fantasy,Action
2                         Action,Adventure,Crime
3                    Action,Crime,Drama,Thriller
4               Action,Adventure,Science Fiction
                          ...                   
4804                       Action,Crime,Thriller
4805                              Comedy,Romance
4806               Comedy,Drama,Romance,TV Movie
4807                                            
4808                                 Documentary
Name: genres, Length: 4806, dtype: object

In [7]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(movies_summary_clean_1)

from sklearn.feature_extraction.text import CountVectorizer

In [8]:
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(","), binary='true')
y_train = vectorizer.fit_transform(train['genres']).toarray()
y_test = vectorizer.transform(test['genres']).toarray()
print(y_train)

[[0 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from tensorflow.keras.layers import Flatten

In [11]:
test

,title,clean_tag,genres
1407,Something Borrowed,though rachel ginnifer goodwin is a successful...,"Comedy,Drama,Romance"
4520,Open Water,two divers are left out at sea without a boat ...,"Drama,Thriller"
1354,The Rite,seminary student michael kovak colin odonoghue...,"Drama,Thriller,Horror"
3839,The Nun's Story,gabrielle van der mal gave up everything to be...,Drama
3531,The Sting,set in the s this intricate caper deals with a...,"Comedy,Crime,Drama"
...,...,...,...
3502,The Greatest,teenagers rose and bennett were in love and th...,Drama
656,Primary Colors,in this adaptation of the best selling roman c...,"Comedy,Drama"
926,Garfield,garfield the fat lazy lasagna lover has everyt...,"Animation,Comedy,Family"
395,The Holiday,two women one cameron diaz from america and on...,"Comedy,Romance"


In [10]:
max_features = 20000 # this is the number of words we care about


vect=Tokenizer(num_words=max_features, split=' ', oov_token='<unw>', filters=' ')
vect.fit_on_texts(train['clean_tag'])
vocab_size = len(vect.word_index) + 1
print(vocab_size)

18855


In [11]:
encoded_docs_train = vect.texts_to_sequences(train['clean_tag'])
max_length = vocab_size


In [14]:
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=175, padding='post')
print(padded_docs_train)

[[1935  669 5379 ...    0    0    0]
 [  59  946    5 ...    0    0    0]
 [ 911  993    9 ...    0    0    0]
 ...
 [4093 3222   60 ...    0    0    0]
 [   2   76 4068 ...    0    0    0]
 [  18    8  218 ...    0    0    0]]


Movie Genre Prediction Using Natural Language Processing (NLP)

In [14]:
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.6 MB 8.3 MB/s 
     |████████████████████████████████| 511.7 MB 5.8 kB/s 
     |████████████████████████████████| 438 kB 45.1 MB/s 
     |████████████████████████████████| 5.8 MB 38.1 MB/s 
     |████████████████████████████████| 1.6 MB 49.7 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninst

In [12]:
train['clean_tag']

587     a civilian oil rig crew is recruited to conduc...
4137    after serving time in prison former drug addic...
2905    in five years after the gamilons began an inva...
3493    a dentist finds himself a murder suspect after...
1084    a young couple living in a connecticut suburb ...
                              ...                        
4516    tilen an attractive high school student swears...
1175    in the antarctic after an expedition with dr d...
3749    theres a deadly zombie epidemic threatening hu...
2482    a teenager journeys through a series of foster...
4665    what s the matter with kathy jordan trovillion...
Name: clean_tag, Length: 3604, dtype: object

In [13]:
import tensorflow as tf
import tensorflow_hub as hub


In [14]:
import tensorflow_text as text

In [15]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def get_sentence_embeding(sentences):
    return bert_encoder(sentences)

get_sentence_embeding(encoded_docs_train)


In [16]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(21, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [17]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

model.compile(optimizer='adam', loss='binary_crossentropy', metrics = METRICS)

In [18]:
model.fit(train['clean_tag'], y_train, epochs=5)

Epoch 1/5
113/113 [==============================] - 1771s 16s/step - loss: 0.3173 - tp: 1163.0000 - fp: 1446.0000 - tn: 65077.0000 - fn: 7998.0000 - accuracy: 0.8752 - precision: 0.4458 - recall: 0.1270 - auc: 0.7793
Epoch 2/5
113/113 [==============================] - 1655s 15s/step - loss: 0.2915 - tp: 1339.0000 - fp: 914.0000 - tn: 65609.0000 - fn: 7822.0000 - accuracy: 0.8846 - precision: 0.5943 - recall: 0.1462 - auc: 0.8200
Epoch 3/5
113/113 [==============================] - 1644s 15s/step - loss: 0.2825 - tp: 1511.0000 - fp: 975.0000 - tn: 65548.0000 - fn: 7650.0000 - accuracy: 0.8860 - precision: 0.6078 - recall: 0.1649 - auc: 0.8351
Epoch 4/5
113/113 [==============================] - 1644s 15s/step - loss: 0.2764 - tp: 1767.0000 - fp: 1015.0000 - tn: 65508.0000 - fn: 7394.0000 - accuracy: 0.8889 - precision: 0.6352 - recall: 0.1929 - auc: 0.8445
Epoch 5/5
113/113 [==============================] - 1638s 14s/step - loss: 0.2712 - tp: 1901.0000 - fp: 1106.0000 - tn: 65417.000

In [19]:
model.evaluate(test['clean_tag'], y_test)

38/38 [==============================] - 542s 14s/step - loss: 0.2671 - tp: 556.0000 - fp: 254.0000 - tn: 21947.0000 - fn: 2485.0000 - accuracy: 0.8915 - precision: 0.6864 - recall: 0.1828 - auc: 0.8624


[0.26710012555122375,
 556.0,
 254.0,
 21947.0,
 2485.0,
 0.8914903998374939,
 0.6864197254180908,
 0.18283459544181824,
 0.8624445199966431]